In [1]:
import os
os.chdir('../')

from extract import open_server_users
from extract import open_server_engage
from extract import open_server_payment
from extract import open_server_battle
from extract import open_server_resource
from extract import open_server_troop
from extract import server_open_nd
from extract import open_server_goldfarmer

from transform import goldfarmer_transform
from pipeline import feature_generation

import TASKS

from data_utils import run_aggregation_tasks,asfreq_by_group

import numpy as np
from config import ETLConfig
from functools import reduce

from camel_utils_x.camel_sql import SQLBase

from extract.utils import rename_aggr, server_open_nd

In [10]:
# task = dict(game = 'aoz',server_ids = [234,235,236],
#             offset = ETLConfig.OFFSET,days =ETLConfig.OPEN_SERVER_DAYS)

# task['server_open'] = server_open_nd(**task)

# server_features = feature_generation(task, ETLConfig)

In [71]:
server_id = 233
days = 30

In [72]:
    server_open = server_open_nd(game = 'aoz', days = 30, offset = 0)

In [184]:
@multiserver
def payment_nd(game = 'aoz', server_id = 233,days = 30,tool = None):
    global server_open
    fields = ['currency_ammount']
    table = 'payment_record'
    parsers = { 'currency_ammount':'sum'}
    alias = ['amount']
    before_date = server_open.loc[server_id,f'open_{days}_date']

    sql = SQLBase(fields = fields, table = table, 
                  parsers = parsers, 
                  alias = alias).\
        stamp13beforeq('currency_ammount', before_date).make()
    
    conn = tool.get_connection(**tool.get_conn_info(server_id,'gs'))
    df = pd.read_sql_query(sql,conn)
    return df

In [185]:
tool = Dbtools.initialize('all','aoz')

In [186]:
payment_nd(game = 'aoz', server_id = [1,2,3,4,5,6,7], tool = None)

,amount,from_server
0,3.880170e+06,1
0,4.780392e+06,2
0,2.602879e+05,3
0,1.444762e+05,4
0,3.816812e+05,5
0,1.429232e+05,6
0,4.838314e+05,7


In [36]:
user_type_col = [col for col in user_features.columns if col.startswith('user_type')]

In [39]:
utype = user_features[user_type_col].reset_index()

In [34]:
utype.reset_index().groupby('from_server')

In [44]:
utype.drop('day_id', axis = 1).groupby('from_server').cumsum()

,user_type_cur_dummy_account_count_user_id,user_type_cur_init_user_count_user_id,user_type_pure_roller_count_user_id,user_type_sandbox_count_user_id
0,3.0,6579.0,148.0,0.0
1,118.0,54715.0,415.0,0.0
2,268.0,68924.0,534.0,0.0
3,339.0,69195.0,581.0,0.0
4,390.0,69239.0,621.0,0.0
5,428.0,69263.0,650.0,0.0
6,45.0,34210.0,285.0,0.0
7,191.0,70826.0,495.0,0.0
8,281.0,70952.0,568.0,0.0
9,341.0,71005.0,623.0,0.0


In [4]:
from camel_queries.query_sso import query_server_open

In [1]:
import os
os.chdir('../')

from targets import payment_nd

In [3]:
payment_nd('aoz',list(range(1,30)),30)

,amount,from_server
0,3.897566e+06,1
0,4.798374e+06,2
0,2.602879e+05,3
0,1.444762e+05,4
0,3.816812e+05,5
0,1.429232e+05,6
0,4.838314e+05,7
0,3.536715e+05,8
0,2.679340e+05,9
0,9.227479e+05,10


In [2]:
print(3+3)

6


In [39]:
from extract.utils import server_open_nd
from camel_utils_x.camel_sql import SQLBase
import pandas as pd
from dbx import Dbtools
from tqdm import tqdm

In [17]:
days = 30

In [18]:
SERVER_OPEN = server_open_nd('aoz',days = 30,offset = 0)

In [28]:
fields = ['currency_ammount']
table = 'payment_record'
parsers = { 'currency_ammount':'sum'}
alias = ['amount']
before_date = SERVER_OPEN.loc[234,f'open_{days}_date']

sql = SQLBase(fields = fields, table = table,
              parsers = parsers,
              alias = alias).\
    stamp13beforeq('currency_ammount', before_date).make()

tool = Dbtools.initialize('all','aoz')

dfs = []
pbar = tqdm(server_ids, desc = 'Battle: ')
for server_id in pbar:
    pbar.set_description('Battle {}'.format(server_id))
    connection = tool.get_connection(**tool.get_conn_info(server_id,'gs'))
    result = pd.read_sql_query(sql, connection)
    result['from_server'] = server_id
    dfs.append(result)
dfs=  pd.concat(dfs, sort = False)

In [45]:
import logging

In [51]:
payment_nd(list(range(1,300)),'aoz',30)

Paymen_nd 299: 100%|████████████████████████████████████| 299/299 [01:39<00:00,  2.99it/s]


,amount,from_server
0,3.897917e+06,1
0,4.798454e+06,2
0,2.602879e+05,3
0,1.444762e+05,4
0,3.816812e+05,5
...,...,...
0,3.606804e+05,295
0,3.567295e+05,296
0,4.705829e+05,297
0,4.992496e+05,298


In [1]:
import os
os.chdir('../')

In [2]:
from targets import payment_nd

In [4]:
payment_nd(game = 'wao',days = 30)

Paymen_nd 1246: 100%|█████████████████████████████████| 1246/1246 [05:29<00:00,  3.78it/s]


,amount,from_server
0,9.601847e+05,1
0,4.703895e+05,2
0,9.467547e+06,3
0,1.485440e+06,4
0,5.495225e+05,5
...,...,...
0,8.538852e+04,1242
0,3.120347e+04,1243
0,5.239799e+04,1244
0,2.925309e+04,1245


In [1]:
from camel_utils_x.camel_sql import SQLBase

In [3]:
parsers = {'distinct id':'count'}

print(SQLBase(fields = ['distinct id'], 
              table = 'user_info', 
              parsers = parsers)..make())


        SELECT count(distinct id)
        FROM user_info
        
        
        
        
        
        


In [1]:
import os
os.chdir('../')

In [2]:
from targets import user_count_nd

In [4]:
user_count_nd(game = 'aoz')

User Count 354: 100%|███████████████████████████████████| 354/354 [00:49<00:00,  7.12it/s]


,register_count,from_server
0,54231,1
0,63840,2
0,31761,3
0,32168,4
0,39742,5
...,...,...
0,91969,350
0,93817,351
0,89174,352
0,89129,353


In [5]:
user_count_nd(game = 'wao')

User Count 1246: 100%|████████████████████████████████| 1246/1246 [07:46<00:00,  2.67it/s]


,register_count,from_server
0,8846,1
0,17811,2
0,28955,3
0,27313,4
0,29659,5
...,...,...
0,59540,1242
0,32483,1243
0,64171,1244
0,31570,1245
